In [1]:
# This notebook contains a naive mlp for garbage classification
%load_ext autoreload
%autoreload 2
import torch
import torch.nn as nn
import utils
from vocab import Vocabulary
import pandas as pd
import numpy as np
import torch.nn.functional as F

In [2]:
import os
glove_path = os.path.expanduser('~/Downloads/glove.6B.100d.txt')
embedding_dict = utils.load_glove_to_dict(glove_path)

400000it [00:10, 36792.65it/s]


In [3]:
complete_csv_path = './preprocessed.csv'
complete_df = pd.read_csv(complete_csv_path)

complete_vocab = utils.build_vocab(complete_df)
print(complete_vocab)

wvecs = utils.build_wvecs(embedding_dict, complete_vocab)

print(len(wvecs))

train_csv_path = './train.csv'
val_csv_path = './val.csv'
test_csv_path = './test.csv'

Vocabulary with 2169 items
2169


In [15]:
# Load dataset
dataset = utils.GarbageDataset('./preprocessed.csv', complete_vocab)
loader = torch.utils.data.DataLoader(dataset, batch_size=32, collate_fn=utils.collate_fn)

In [16]:
class baseline_mlp(nn.Module):
    def __init__(self, embedding_dim, vocab_size):
        super(baseline_mlp, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        self.fc1 = nn.Linear(embedding_dim * 10, 100)
        self.fc2 = nn.Linear(100, 10)
        self.out = nn.Linear(10, 4)
    
    def forward(self, sequence):
        batch_size = sequence.shape[0]
        embedded = self.embedding(sequence)
        embedded = embedded.view(batch_size, -1)
        z = self.fc1(embedded)
        z = self.fc2(z)
        z = self.out(z)
        return z


In [21]:

use_cuda = True
device = torch.device("cuda:0" if use_cuda else "cpu") 

mlp = baseline_mlp(100, len(complete_vocab)).to(device)
mlp.embedding.weight.data.copy_(torch.from_numpy(np.array(wvecs)))
max_epoch = 100

optimizer = torch.optim.Adam(mlp.parameters(), lr=5e-4)

loss_fn = nn.CrossEntropyLoss()
loss_fn.to(device)

def eval(loader, model):
    mean_acc = 0
    correct_counts = 0
    with torch.no_grad():
        for batch in loader:
            input, target = batch
            input, target = input.to(device), target.to(device)
            pred = mlp(input).squeeze(1)
            pred_classes = torch.argmax(pred, dim=1)
            correct_counts += torch.sum(pred_classes == target)
    print(correct_counts.item() / len(loader))




print('Start training')
for epoch in range(max_epoch):
    mlp.train()
    epoch_loss = 0
    for batch in loader:
        optimizer.zero_grad()
        input, target = batch
        input, target = input.to(device), target.to(device)
        pred = mlp(input).squeeze(1)
        loss = loss_fn(pred, target.long())
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item() * target.shape[0]
    eval(loader, mlp)
    print('Epoch {} loss {}'.format(epoch, epoch_loss))


Start training
7.508064516129032
Epoch 0 loss 6612.10493535921
3.056451612903226
Epoch 1 loss 6127.971958398819
6.838709677419355
Epoch 2 loss 5439.284460067749
5.830645161290323
Epoch 3 loss 5258.055226325989
6.556451612903226
Epoch 4 loss 5117.64919757843
6.75
Epoch 5 loss 4960.16169834137
6.57258064516129
Epoch 6 loss 4763.5505294799805
6.169354838709677
Epoch 7 loss 4589.196720123291
6.548387096774194
Epoch 8 loss 4270.767127037048
8.78225806451613
Epoch 9 loss 4030.767740249634
11.983870967741936
Epoch 10 loss 3678.102199077606
15.60483870967742
Epoch 11 loss 3321.703118801117
18.201612903225808
Epoch 12 loss 2968.5224742889404
19.975806451612904
Epoch 13 loss 2628.9176198244095
21.379032258064516
Epoch 14 loss 2341.1288553476334
22.443548387096776
Epoch 15 loss 2095.168779551983
23.475806451612904
Epoch 16 loss 1886.1227180361748
24.161290322580644
Epoch 17 loss 1706.5889381170273
24.846774193548388
Epoch 18 loss 1550.9148988127708
25.54032258064516
Epoch 19 loss 1415.65189030766